## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    roc_curve,
    auc,
    roc_auc_score
)
from sklearn.preprocessing import label_binarize
import joblib
import json
import warnings
warnings.filterwarnings('ignore')

print("Packages importes")

## Chargement du meilleur modele

In [2]:
X_train = np.load('data/X_train_scaled.npy')
X_test = np.load('data/X_test_scaled.npy')
y_train = np.load('data/y_train.npy')
y_test = np.load('data/y_test.npy')
le = joblib.load('models/label_encoder.pkl')

with open('results/tuning/best_model_info.json', 'r') as f:
    best_model_info = json.load(f)

model_name = best_model_info['model_name']
model_filename = f"models/tuned/{model_name.replace(' ', '_').lower()}_best.pkl"
best_model = joblib.load(model_filename)

n_classes = len(le.classes_)

print(f"Meilleur modele: {model_name}")
print(f"Test Accuracy: {best_model_info['test_score']:.4f}")
print(f"CV Score: {best_model_info['cv_score']:.4f}")
print(f"\nNombre de classes: {n_classes}")
print(f"Classes: {le.classes_}")

# Courbe overfitting 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.base import clone

# ===============================
# Courbe d'overfitting (réelle) pour le meilleur modèle
# ===============================

# 1) On fait varier la "taille effective" du training set
train_sizes_frac = np.linspace(0.1, 1.0, 10)   # de 10% à 100%

train_scores = []
test_scores = []

for frac in train_sizes_frac:
    n_samples = int(len(X_train) * frac)
    X_sub = X_train[:n_samples]
    y_sub = y_train[:n_samples]

    # On clone le meilleur modèle (Naive Bayes) pour ne pas le modifier
    model_clone = clone(best_model)
    model_clone.fit(X_sub, y_sub)

    # Accuracy sur train (subset) et sur test (complet)
    y_pred_train = model_clone.predict(X_sub)
    y_pred_test  = model_clone.predict(X_test)

    train_scores.append(accuracy_score(y_sub, y_pred_train))
    test_scores.append(accuracy_score(y_test, y_pred_test))

train_scores = np.array(train_scores)
test_scores  = np.array(test_scores)

# 2) On passe en erreur = 1 - accuracy (comme les schémas théoriques)
train_errors = 1 - train_scores
test_errors  = 1 - test_scores

# Position du "good-fit" = là où l'erreur test est minimale
idx_opt = np.argmin(test_errors)
x_opt = train_sizes_frac[idx_opt] * 100      # en %

plt.figure(figsize=(8, 5))

plt.plot(train_sizes_frac * 100, train_errors,
         marker="o", linewidth=2, label="Training error")
plt.plot(train_sizes_frac * 100, test_errors,
         marker="o", linewidth=2, label="Validation/Test error")

# Ligne verticale Good-fit
plt.axvline(x_opt, color="black", linestyle="--")
plt.text(x_opt, max(train_errors.max(), test_errors.max()) * 0.98,
         "Good-fit", ha="center", va="top")

# Zones Under-fitting / Over-fitting (texte)
plt.text(15, max(train_errors.max(), test_errors.max()) * 0.9,
         "Under-fitting", ha="center")
plt.text(85, max(train_errors.max(), test_errors.max()) * 0.9,
         "Over-fitting", ha="center")

plt.xlabel("Training set size (%)")
plt.ylabel("Error (1 - accuracy)")
plt.title("Under-fitting vs Over-fitting – Best Model (Naive Bayes)")
plt.legend(loc="upper right")
plt.grid(True)
plt.tight_layout()
plt.show()


## Predictions

In [4]:
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

if hasattr(best_model, 'predict_proba'):
    y_test_proba = best_model.predict_proba(X_test)
elif hasattr(best_model, 'decision_function'):
    y_test_proba = best_model.decision_function(X_test)
else:
    y_test_proba = None

print(f"Predictions generees")
print(f"Train predictions: {y_train_pred.shape}")
print(f"Test predictions: {y_test_pred.shape}")
if y_test_proba is not None:
    print(f"Test probabilities: {y_test_proba.shape}")

## Metriques detaillees

In [5]:
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

precision_macro = precision_score(y_test, y_test_pred, average='macro', zero_division=0)
precision_weighted = precision_score(y_test, y_test_pred, average='weighted', zero_division=0)

recall_macro = recall_score(y_test, y_test_pred, average='macro', zero_division=0)
recall_weighted = recall_score(y_test, y_test_pred, average='weighted', zero_division=0)

f1_macro = f1_score(y_test, y_test_pred, average='macro', zero_division=0)
f1_weighted = f1_score(y_test, y_test_pred, average='weighted', zero_division=0)

metrics_summary = {
    'Train Accuracy': train_acc,
    'Test Accuracy': test_acc,
    'Precision (Macro)': precision_macro,
    'Precision (Weighted)': precision_weighted,
    'Recall (Macro)': recall_macro,
    'Recall (Weighted)': recall_weighted,
    'F1-Score (Macro)': f1_macro,
    'F1-Score (Weighted)': f1_weighted
}

print(f"METRIQUES - {model_name}")
print("="*60)
for metric, value in metrics_summary.items():
    print(f"{metric:25s}: {value:.4f}")

metrics_df = pd.DataFrame([metrics_summary])
metrics_df.insert(0, 'Model', model_name)
metrics_df.to_csv('results/metrics/best_model_metrics.csv', index=False)
print("\nSauvegarde: results/metrics/best_model_metrics.csv")

## Matrice de confusion

In [6]:
cm = confusion_matrix(y_test, y_test_pred)

fig, ax = plt.subplots(figsize=(14, 12))
im = ax.imshow(cm, cmap='YlGnBu', aspect='auto', interpolation='nearest')

ax.set_xticks(np.arange(n_classes))
ax.set_yticks(np.arange(n_classes))
ax.set_xticklabels(le.classes_, rotation=45, ha='right')
ax.set_yticklabels(le.classes_)

for i in range(n_classes):
    for j in range(n_classes):
        color = 'white' if cm[i, j] > cm.max() / 2 else 'black'
        ax.text(j, i, str(cm[i, j]), ha='center', va='center', color=color, fontsize=10)

ax.set_title(f'Confusion Matrix - {model_name}\nTest Accuracy: {test_acc:.4f}', fontsize=16)
ax.set_xlabel('Predicted Crop', fontsize=12)
ax.set_ylabel('Actual Crop', fontsize=12)
cbar = fig.colorbar(im, ax=ax)

plt.tight_layout()
plt.savefig(f'results/visualizations/confusion_matrix_{model_name.replace(" ", "_").lower()}.png',
            dpi=300, bbox_inches='tight')
plt.show()

print(f"Sauvegarde: results/visualizations/confusion_matrix_{model_name.replace(' ', '_').lower()}.png")

## Classification Report

In [7]:
print(f"CLASSIFICATION REPORT - {model_name}")
print("="*80)
print(classification_report(y_test, y_test_pred, target_names=le.classes_))

report_dict = classification_report(y_test, y_test_pred, target_names=le.classes_, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv(f'results/metrics/classification_report_{model_name.replace(" ", "_").lower()}.csv')

print(f"\nSauvegarde: results/metrics/classification_report_{model_name.replace(' ', '_').lower()}.csv")

## Visualisation des metriques

In [8]:
fig = plt.figure(figsize=(16, 10))

metrics_names = ['Precision (Weighted)', 'Recall (Weighted)', 'F1-Score (Weighted)', 'Test Accuracy']
metrics_values = [precision_weighted, recall_weighted, f1_weighted, test_acc]
colors = ['lightcoral', 'lightgreen', 'plum', 'skyblue']

for idx, (metric_name, value, color) in enumerate(zip(metrics_names, metrics_values, colors)):
    ax = fig.add_subplot(2, 2, idx + 1)
    ax.barh([model_name], [value], color=color, edgecolor='black', height=0.5)
    ax.set_xlim([0, 1])
    ax.set_xlabel(metric_name, fontsize=12)
    ax.set_title(f'{metric_name}: {value:.4f}', fontsize=14)
    ax.grid(axis='x', alpha=0.3)
    ax.text(value + 0.02, 0, f'{value:.4f}', va='center', fontsize=11)

plt.tight_layout()
plt.savefig('results/visualizations/best_model_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("Sauvegarde: results/visualizations/best_model_metrics.png")

## Courbes ROC (One-vs-Rest)

In [9]:
if y_test_proba is not None:
    y_test_bin = label_binarize(y_test, classes=range(n_classes))
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_test_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_test_proba.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    plt.figure(figsize=(12, 8))
    
    plt.plot(fpr["micro"], tpr["micro"],
             label=f'Micro-average (AUC = {roc_auc["micro"]:.3f})',
             color='deeppink', linestyle=':', linewidth=3)
    
    colors = plt.cm.tab20(np.linspace(0, 1, n_classes))
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'{le.classes_[i]} (AUC = {roc_auc[i]:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontweight='bold', fontsize=12)
    plt.ylabel('True Positive Rate', fontweight='bold', fontsize=12)
    plt.title(f'ROC Curves - {model_name}', fontsize=16, fontweight='bold')
    plt.legend(loc="lower right", fontsize=9)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    
    plt.savefig(f'results/visualizations/roc_curves_{model_name.replace(" ", "_").lower()}.png',
                dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Sauvegarde: results/visualizations/roc_curves_{model_name.replace(' ', '_').lower()}.png")
else:
    print("Pas de probabilites disponibles pour les courbes ROC")

## AUC Scores

In [10]:
if y_test_proba is not None:
    try:
        y_test_bin = label_binarize(y_test, classes=range(n_classes))
        
        auc_macro = roc_auc_score(y_test_bin, y_test_proba, average='macro', multi_class='ovr')
        auc_weighted = roc_auc_score(y_test_bin, y_test_proba, average='weighted', multi_class='ovr')
        
        auc_per_class = []
        for i in range(n_classes):
            auc_class = roc_auc_score(y_test_bin[:, i], y_test_proba[:, i])
            auc_per_class.append({
                'Class': le.classes_[i],
                'AUC': auc_class
            })
        
        auc_class_df = pd.DataFrame(auc_per_class)
        
        print("AUC SCORES")
        print("="*60)
        print(f"AUC (Macro):    {auc_macro:.4f}")
        print(f"AUC (Weighted): {auc_weighted:.4f}")
        print("\nAUC par classe:")
        print(auc_class_df.to_string(index=False))
        
        auc_summary = {
            'Model': model_name,
            'AUC (Macro)': auc_macro,
            'AUC (Weighted)': auc_weighted
        }
        auc_summary_df = pd.DataFrame([auc_summary])
        auc_summary_df.to_csv('results/metrics/best_model_auc.csv', index=False)
        
        auc_class_df.to_csv('results/metrics/best_model_auc_per_class.csv', index=False)
        
        print("\nSauvegarde: results/metrics/best_model_auc.csv")
        print("Sauvegarde: results/metrics/best_model_auc_per_class.csv")
        
    except Exception as e:
        print(f"Erreur calcul AUC: {e}")
else:
    print("Pas de probabilites disponibles pour le calcul AUC")

## AUC par classe (visualisation)

In [11]:
if y_test_proba is not None and len(auc_per_class) > 0:
    plt.figure(figsize=(14, 8))
    
    auc_class_df_sorted = auc_class_df.sort_values('AUC', ascending=True)
    
    colors_gradient = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(auc_class_df_sorted)))
    
    plt.barh(auc_class_df_sorted['Class'], auc_class_df_sorted['AUC'],
             color=colors_gradient, edgecolor='black')
    
    plt.xlabel('AUC Score', fontweight='bold', fontsize=12)
    plt.ylabel('Crop Class', fontweight='bold', fontsize=12)
    plt.title(f'AUC Score per Class - {model_name}', fontsize=16, fontweight='bold')
    plt.xlim([0, 1])
    plt.grid(axis='x', alpha=0.3)
    
    for idx, (crop, auc_val) in enumerate(zip(auc_class_df_sorted['Class'], auc_class_df_sorted['AUC'])):
        plt.text(auc_val + 0.01, idx, f'{auc_val:.3f}', va='center', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('results/visualizations/auc_per_class.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("Sauvegarde: results/visualizations/auc_per_class.png")

## Resume final

In [12]:
print("\n" + "="*70)
print(" RESUME EVALUATION")
print("="*70)
print(f"\nModele: {model_name}")
print(f"\nHyperparametres optimaux:")
for param, value in best_model_info['best_params'].items():
    print(f"  {param}: {value}")

print(f"\nPerformances:")
print(f"  Train Accuracy: {train_acc:.4f}")
print(f"  Test Accuracy:  {test_acc:.4f}")
print(f"  CV Score:       {best_model_info['cv_score']:.4f}")
print(f"  Precision (W):  {precision_weighted:.4f}")
print(f"  Recall (W):     {recall_weighted:.4f}")
print(f"  F1-Score (W):   {f1_weighted:.4f}")

if y_test_proba is not None:
    print(f"  AUC (Macro):    {auc_macro:.4f}")
    print(f"  AUC (Weighted): {auc_weighted:.4f}")

print(f"\nFichiers sauvegardes:")
print(f"  - results/metrics/best_model_metrics.csv")
print(f"  - results/metrics/classification_report_{model_name.replace(' ', '_').lower()}.csv")
if y_test_proba is not None:
    print(f"  - results/metrics/best_model_auc.csv")
    print(f"  - results/metrics/best_model_auc_per_class.csv")
print(f"  - results/visualizations/confusion_matrix_{model_name.replace(' ', '_').lower()}.png")
print(f"  - results/visualizations/best_model_metrics.png")
if y_test_proba is not None:
    print(f"  - results/visualizations/roc_curves_{model_name.replace(' ', '_').lower()}.png")
    print(f"  - results/visualizations/auc_per_class.png")

print("\n" + "="*70)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=37cd5642-eb00-4cb1-969e-a9bc85cf5e83' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>